<h3 style="text-decoration: underline"><center>Brief Project : Simplon Search</h3>

In [44]:
import re 
from re import *

<h3 style="text-decoration: underline">1.Trouver toutes les occurences d'un mot dont une lettre peut manquer</h3>

In [45]:
# This function takes a sentence s and a word s and returns a list of word existing in s.
# The word could be exactly the same as given or could have one letter missing
def almost(word, s): 
    
    # We replace commas with empty string to dissociate the word from the commas
    s = s.replace(",","")
    
    # We replace the last dot with a space to dissociate the word from the dot. 
    # Furthermore, we will consider a word if it has a space before its first letter and a space after its last letter
    s = s.replace("."," ")
    s = " " +s
    
    
    # We search for all exact word (starting and ending with a space) present in the s string.
    found = findall("\s"+word+"\s", s)
    
    # To deal with words that may lack one letter we pass the word through a for loop 
    for i in range(len(word)): 
        
        # The pattern has a space before and after its first and last letter
        # The possible patterne could be all those words missing a letter at any i position
        pattern = " " + word[:i] + word[i+1:] + " " 
        
        # We increment the list of exact word found before the for loop with the list of approximative words (lacking one letter)
        found += findall(pattern, s)
        
    # We print all occurences of found pattern    
    print(found)

In [46]:
s= "Les etrois tris, lys trois gros, les troisx roi."
word = "trois"

In [47]:
almost(word,s)

[' trois ', ' tris ']


<h3 style="text-decoration: underline">2.Trouver toutes les occurences d'un mot dont une lettre peut être remplacée, ajoutée ou manquée</h3>

In [62]:
# This function search for different occurences of the word in s. 
# The word could be exactly the same, or could have one letter added at the end, 
# or one letter missing, or one extra letter inside the word. 
# It returns a list of word present in s that match the word approximately
def larger(word, s):
    
    # We add a space before the start and one after the end of the sentence
    # and replace dots and commas with an empty string 
    s = " " + sub("[.,]", "", s) + " "
    
    # We search for all words in s containing the first argument pattern
    # We look at all words starting with a white space and finishing with a white space 
    # and that are strictly specific to the word. The result will be output in a list
    # By adding parenthesis at this place the output doesn't consider the surrounding white space
    # but only the word
    # EXACT WORD
    found = findall("\s(" + word + ")\s", s)
    
    # Then we append to the list the result of finding in s all the pattern that sart and finish with a white space
    # The core pattern is made of the word that joins any single character which is not a white space
    # WORD WITH AN EXTRA LETTER AT THE END
    found +=findall('\s('+word+"[^\s]"+")\s",s)
    
    # Then we look for words with an EXTRA LETTER INSIDE the word 
    # OR a word with a LETTER MISSING INSIDE the word 
    # To do this we have to pass by a for loop to consider each letter position
    for i in range(len(word)):
        
        # This will look for a word with possibly the ith letter that could be present 0 or 1 time (?)
        # Brackets position is important too.
        found += findall("\s(" + word[:i] + "[^" + word[i] + "]?" + word[i + 1:] + ")\s", s)
        
        # This will look for a word with any single character but not a white space added before the ith letter of the word
        found += findall("\s(" + word[:i] + "[^\s]" + word[i:] + ")\s", s)
    return found

In [ ]:
s= "Les etrois tris, lys trois gros, les troisx roi."
word = "trois"

In [65]:
larger(word,s)

['trois', 'troisx', 'etrois', 'tris']

<h3 style="text-decoration: underline">3.Établir un score en fonction de la présence et de l'exactitude des mots</h3>

In [76]:
# This function takes a sentence p (phrase) and gives it a score depending on word presence in another sentence s. 
# 5 points for an exact word, 1 point for an approximated word. 
def score(p,s):
    
    # We initialise a score variable to 0 
    score = 0 
    
    # We deal with entry sentences p and s to add white space before and after each sentence
    # We also replace dots and commas with empty string 
    s = " " + sub("[.,]", "", s) + " "
    p = " " + sub("[.,]", "", p) + " "

    # We split the p sentence
    # Returns a list of word
    p_list = p.split()
    
    # We pass by a for loop to check if each word in p could be present in s
    for word in p_list: 
        
        # We check for all the possible words by applying the previous larger function
        list_found = larger(word,s)
        
        # If the word is in a the p_list and in the list_found
        # Then we add 5 if it is an exact word, and 1 if it is an approximated one
        for words in list_found:  
            score += 5 if words in p else 1
                
    return score 

In [77]:
p = "Le petit bonhomme en mousse"
s = "Ce superbe matelas en mousse naturelle."

score(p,s)

11

<h3 style="text-decoration: underline">4.Donner un bonus si deux mos successifs sont repérés</h3>

In [87]:
# This function is derived from the previous. 
# It just add 20 to the score if two words are successive in p and s. 
def score2(p,s):
    
    score = 0 
    
    s = " " + sub("[.,]", "", s) + " "
    p = " " + sub("[.,]", "", p) + " "

    # We split the p sentence
    # Returns a list of word
    p_list = p.split()
    
    # We pass by a for loop to check if each word in p could be present in s
    for word in p_list: 
        
        # We check for all the possible words by applying the previous larger function 
        # If the word is in a the p_list and in the list_found
        # Then we add 5 if it is an exact word, and 1 if it is an approximated one
        for words in larger(word, s):  
            score += 5 if words in p else 1
            
        # We check if two successive words in p appears samely in s
        # If so, we add 20 to the score
        for x in range(len(p_list)-1): 
            if " "+ p_list[x] + " " + p_list[x+1] + " " in s: 
                score += 20
                
    return score 

In [90]:
p = "Le petit bonhomme en mousse"
s = "Ce superbe matelas en mousse naturelle."

score2(p,s)

111

<h3 style="text-decoration: underline">5.Application sur un ensemble de fichiers</h3>

In [97]:
# This code applies previous functions on a set of text files available in text_folder
# We must give output a score and the higher it is, the more the file related to that score 
# has the corresponding search in it

# We import the os module to be able to work with file and directories
import os 

# os.listdir() returns a list containing name of every file in the folder
files_list = os.listdir("text_folder")

# We initialise a empty results variable as a dictionnary that will contain
# key(text file name) : value(score associated) pairs
results = {}

# The search pattern we are looking for in the text files 
search = "analyse et visualisation"

# We loop over each file in files_list
for file in files_list :
    
    # We open the file 
    current_file = open("text_folder/"+file, "r", encoding = "utf-8")
    
    # We read the file 
    current_file = current_file.read()
    
    # We apply the score function to the file, searching for the search pattern
    results[file] = score(search, current_file)

# Finally we sort the dictionnary, we apply a lambda function to sort it by the value 
final = sorted(results.items(),key=lambda item: item[1], reverse = True)

# We do a for loop to print files name and their score. 
for i, j in final: 
    print(i + " : " + str(j))

Matplotlib.txt : 75
Pandas.txt : 71
Seaborn.txt : 40
Numpy.txt : 38
